In [1]:
!pip install chromadb

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tomli-2.2.1-py3-none-any.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 1.1 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 1.8 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 1.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 2.6 MB/s eta 0:00:00a 0:00:01
Using cached tomli-2.2.1-py3-none-any.whl (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 3.8 MB/s eta 0:00:00a 0:00:01
  Created wheel for pypika: filename=pypika-0.48.9-py2.py3-none-any.whl size=53803 sha256=aaa2394c8fe7959f2a3b3a0c98870e79da9a1338b4334e1d48d468195809fab3
  Stored in directory: /Users/Aksorn_AI/Library/Caches/pip/wheels/f7/02/64/d541eac67ec45

In [385]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re, os
from langchain.schema import Document

path = '/Users/Aksorn_AI/Desktop/RAG/data/hotlab-v1.txt'
# path = '/Users/Aksorn_AI/Desktop/RAG/data/protocal-v1.txt'
# path = '/Users/Aksorn_AI/Desktop/RAG/data/iodine-v1.txt'


filename = os.path.basename(path) 

with open(path, 'r', encoding='utf8') as f:
    raw_text = f.read()
print(raw_text)


def set_topic(raw_text):
    topic_pattern = r"^(\d+\.\s.+?:\s*)$"
    topics = re.findall(topic_pattern, raw_text, re.MULTILINE)
    print("topic:", topics)
    return topics


def set_sub_topics(raw_text):
    sub_topic_pattern = r"^(\d+\.\d+\)\s.+?:\s*)$"
    sub_topics = re.findall(sub_topic_pattern, raw_text, re.MULTILINE)
    print("sub_topic:", sub_topics)
    return sub_topics

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,          
    chunk_overlap=0,         
    separators=["\n\n"]      
)

chunks = text_splitter.split_text(raw_text)
topics = set_topic(raw_text)

docs = []
last_sub_topic = None

for i, chunk in enumerate(chunks):
    sub_topics = set_sub_topics(chunk)  # สมมติ return เป็น list หรือ string (กรณีเดียวก็ [str])

    # ถ้าเจอ sub_topic ใหม่ → เก็บไว้
    if sub_topics and len(sub_topics) > 0:
        last_sub_topic = sub_topics
    else:
        # ถ้าไม่เจอ → ใช้อันก่อนหน้า
        sub_topics = last_sub_topic


    doc = Document(
        page_content=chunk,
        metadata={
            "source": path,
            "filename": filename,
            "topic": topics,            # ถ้าอยาก inherit เหมือน sub_topic ก็ปรับ pattern นี้ได้เลย
            "sub-topic": sub_topics,
            "ids": i
        }
    )
    docs.append(doc)



1. การเตรียมสารเภสัชรังสี หรือ Hot lab: 
ทุกเคสการตรวจ ก่อนคำนวนสารเภสัชรังสีต้องดูน้ำหนักของคนไข้ก่อน ซึ่งแต่ละเคสการตรวจสามาถแบ่งตามชื่อการตรวจได้ดังนี้

1.1) การตรวจ Parathyroi: 
ใช้สารเภสัชรังสีเทคนีเชียม(Tco4-MIBI) ปริมาณ 20 mCi 

1.2) การตรวจ  BONE:
ใช้สารเภสัชรังสีเทคนีเชียม(Tco4-MDP) ปริมาณ 20 mCi สำหรับหมอส่วนใหญ่
ใช้ปริมาณนี้กับคนไข้ทุกคน  ยกเว้นหมอสมพรที่ใช้สารเภสัชรังสีเทคนีเชียม(Tco4-MDP) ปริมาณ 15 mCi  
ข้อควรระวัง: ในเคสการตรวจ  BONE สำหรับเด็กอายุต่ำกว่า 18 ปี ต้องเตรียมสารเภสัชรังสีเทคนีเชียม(Tco4-MDP)  โดยการคำนวนปริมาณสารเภสัชรังสีตามน้ำหนักเท่านั้น


topic: ['1. การเตรียมสารเภสัชรังสี หรือ Hot lab: ']
sub_topic: []
sub_topic: ['1.1) การตรวจ Parathyroi: ']
sub_topic: ['1.2) การตรวจ  BONE:']


In [386]:
def flatten_metadata(meta):
    for k, v in meta.items():
        if isinstance(v, list):
            meta[k] = ' | '.join(str(i).strip() for i in v)
    return meta

for doc in docs:
    doc.metadata = flatten_metadata(doc.metadata)
    print(doc.metadata)

{'source': '/Users/Aksorn_AI/Desktop/RAG/data/hotlab-v1.txt', 'filename': 'hotlab-v1.txt', 'topic': '1. การเตรียมสารเภสัชรังสี หรือ Hot lab:', 'sub-topic': None, 'ids': 0}
{'source': '/Users/Aksorn_AI/Desktop/RAG/data/hotlab-v1.txt', 'filename': 'hotlab-v1.txt', 'topic': '1. การเตรียมสารเภสัชรังสี หรือ Hot lab:', 'sub-topic': '1.1) การตรวจ Parathyroi:', 'ids': 1}
{'source': '/Users/Aksorn_AI/Desktop/RAG/data/hotlab-v1.txt', 'filename': 'hotlab-v1.txt', 'topic': '1. การเตรียมสารเภสัชรังสี หรือ Hot lab:', 'sub-topic': '1.2) การตรวจ  BONE:', 'ids': 2}


In [387]:
docs

[Document(metadata={'source': '/Users/Aksorn_AI/Desktop/RAG/data/hotlab-v1.txt', 'filename': 'hotlab-v1.txt', 'topic': '1. การเตรียมสารเภสัชรังสี หรือ Hot lab:', 'sub-topic': None, 'ids': 0}, page_content='1. การเตรียมสารเภสัชรังสี หรือ Hot lab: \nทุกเคสการตรวจ ก่อนคำนวนสารเภสัชรังสีต้องดูน้ำหนักของคนไข้ก่อน ซึ่งแต่ละเคสการตรวจสามาถแบ่งตามชื่อการตรวจได้ดังนี้'),
 Document(metadata={'source': '/Users/Aksorn_AI/Desktop/RAG/data/hotlab-v1.txt', 'filename': 'hotlab-v1.txt', 'topic': '1. การเตรียมสารเภสัชรังสี หรือ Hot lab:', 'sub-topic': '1.1) การตรวจ Parathyroi:', 'ids': 1}, page_content='1.1) การตรวจ Parathyroi: \nใช้สารเภสัชรังสีเทคนีเชียม(Tco4-MIBI) ปริมาณ 20 mCi'),
 Document(metadata={'source': '/Users/Aksorn_AI/Desktop/RAG/data/hotlab-v1.txt', 'filename': 'hotlab-v1.txt', 'topic': '1. การเตรียมสารเภสัชรังสี หรือ Hot lab:', 'sub-topic': '1.2) การตรวจ  BONE:', 'ids': 2}, page_content='\n\n1.2) การตรวจ  BONE:\nใช้สารเภสัชรังสีเทคนีเชียม(Tco4-MDP) ปริมาณ 20 mCi สำหรับหมอส่วนใหญ่\nใช้ปริม

Embedding

In [336]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_function = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base")

Add data to collection

In [286]:
col_name = 'iodine_collection'

In [287]:
from langchain_community.vectorstores import Chroma

vector_store = Chroma(
    persist_directory="/Users/Aksorn_AI/Desktop/RAG/chroma_langchain",
    embedding_function=embedding_function,
    collection_name=col_name
)

vector_store.add_documents(docs)

['a8142970-ce32-408f-9b32-bdd634131036',
 '491c4a0f-eeab-4268-b3dd-bc62131a01d1',
 'eba02d35-b3f9-408e-9a1b-a043c36866bc']

Manage Collection

In [280]:
import chromadb

In [288]:
print(chroma_client.list_collections())

[Collection(name=protocol_collection), Collection(name=hotlab_collection), Collection(name=iodine_collection)]


Delete Collection

In [273]:
# chroma_client.delete_collection(col_name)

Check Collection

In [289]:
# Local persistent database
chroma_client = chromadb.PersistentClient(path="/Users/Aksorn_AI/Desktop/RAG/chroma_langchain")
collection = chroma_client.get_collection(col_name)
data = collection.get(include=["documents", "metadatas"])

for idx, (doc, meta, id_) in enumerate(zip(data["documents"], data["metadatas"], data["ids"])):
    print("="*60)
    print(f"Doc #{idx}, ID: {id_}")
    print("CONTENT:\n", doc)
    print("META:\n", meta)


Doc #0, ID: a8142970-ce32-408f-9b32-bdd634131036
CONTENT:
 1. การให้น้ำแร่หรือการกลืนแร่ Iodine:
META:
 {'ids': 0, 'filename': 'hotlab-v1.txt', 'sub-topic': '1.1) การให้น้ำแร่หรือการกลืนแร่ปริมาณต่ำ(Low dose):', 'topic': '1. การให้น้ำแร่หรือการกลืนแร่ Iodine:', 'source': 'data/hotlab-v1.txt'}
Doc #1, ID: 491c4a0f-eeab-4268-b3dd-bc62131a01d1
CONTENT:
 1.1) การให้น้ำแร่หรือการกลืนแร่ปริมาณต่ำ(Low dose):
ความรู้และข้อมูลการให้น้ำแร่ ( Iodine-131 ) แบบ Low dose และ High dose ให้แก่ผู้ป่วย  
การเตรียมตัวก่อนกลืนแร่ 
ผู้ป่วยต้องเจาะเลือดและพบแพทย์ก่อนได้รับการกลืนแร่ ผลเลือด TSH > 30  จึงจะสามารถกลืนแร่ได้ ( ในกรณีที่ผู้ป่วยเป็น  GD แพทย์อาจจะพิจารณาให้ผู้ป่วยสามารถกลืนแร่ได้ ) 
ผู้ป่วยต้อง NPO อย่างน้อย 1 ชั่วโมงก่อนกลืนแร่ ยกเว้นแพทย์พงศธร NPO อย่างน้อย 2 ชั่วโมง 
ปริมาณแร่  Low dose มีค่าตั้งแต่ 3-30 mCi  โดยเราต้องทำการแนะนำวิธีการปฏิบัติตัวหลังกลืนแร่ ดังนี้ 
อธิบายคนไข้ Low dose 🫧✨🥹
อธิบายการเตรียมตัวคนไข้ บอกว่าแต่ละคนกินเท่าไหร่ 
ให้เซ็นยินยอม 
และพามาดูห้องกินน้ำแร่  
2 สัปดาห์ถ้า 2

In [290]:
retriever = vector_store.as_retriever(
search_type="similarity",
# search_type="mmr",
search_kwargs={
        "k": 5,
        # "fetch_k": 20,
        # "filter": {
        #     "sub-topic": "" 
        # }
    }
)
# Retrieval
query = "การให้น้ำแร่หรือการกลืนแร่ปริมาณต่ำ"
docs = retriever.get_relevant_documents(query)
docs


/var/folders/54/xm1kjlcx7y583vg073fv0z8c0000gn/T/ipykernel_34884/273241079.py:14: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


[Document(metadata={'sub-topic': '1.1) การให้น้ำแร่หรือการกลืนแร่ปริมาณต่ำ(Low dose):', 'source': 'data/hotlab-v1.txt', 'filename': 'hotlab-v1.txt', 'ids': 0, 'topic': '1. การให้น้ำแร่หรือการกลืนแร่ Iodine:'}, page_content='1. การให้น้ำแร่หรือการกลืนแร่ Iodine:'),
 Document(metadata={'source': 'data/hotlab-v1.txt', 'filename': 'hotlab-v1.txt', 'topic': '1. การให้น้ำแร่หรือการกลืนแร่ Iodine:', 'sub-topic': '1.1) การให้น้ำแร่หรือการกลืนแร่ปริมาณต่ำ(Low dose):', 'ids': 1}, page_content='1.1) การให้น้ำแร่หรือการกลืนแร่ปริมาณต่ำ(Low dose):\nความรู้และข้อมูลการให้น้ำแร่ ( Iodine-131 ) แบบ Low dose และ High dose ให้แก่ผู้ป่วย  \nการเตรียมตัวก่อนกลืนแร่ \nผู้ป่วยต้องเจาะเลือดและพบแพทย์ก่อนได้รับการกลืนแร่ ผลเลือด TSH > 30  จึงจะสามารถกลืนแร่ได้ ( ในกรณีที่ผู้ป่วยเป็น  GD แพทย์อาจจะพิจารณาให้ผู้ป่วยสามารถกลืนแร่ได้ ) \nผู้ป่วยต้อง NPO อย่างน้อย 1 ชั่วโมงก่อนกลืนแร่ ยกเว้นแพทย์พงศธร NPO อย่างน้อย 2 ชั่วโมง \nปริมาณแร่  Low dose มีค่าตั้งแต่ 3-30 mCi  โดยเราต้องทำการแนะนำวิธีการปฏิบัติตัวหลังกลืน

In [291]:
context = "\n\n".join(doc.page_content for doc in docs)
print(context)

1. การให้น้ำแร่หรือการกลืนแร่ Iodine:

1.1) การให้น้ำแร่หรือการกลืนแร่ปริมาณต่ำ(Low dose):
ความรู้และข้อมูลการให้น้ำแร่ ( Iodine-131 ) แบบ Low dose และ High dose ให้แก่ผู้ป่วย  
การเตรียมตัวก่อนกลืนแร่ 
ผู้ป่วยต้องเจาะเลือดและพบแพทย์ก่อนได้รับการกลืนแร่ ผลเลือด TSH > 30  จึงจะสามารถกลืนแร่ได้ ( ในกรณีที่ผู้ป่วยเป็น  GD แพทย์อาจจะพิจารณาให้ผู้ป่วยสามารถกลืนแร่ได้ ) 
ผู้ป่วยต้อง NPO อย่างน้อย 1 ชั่วโมงก่อนกลืนแร่ ยกเว้นแพทย์พงศธร NPO อย่างน้อย 2 ชั่วโมง 
ปริมาณแร่  Low dose มีค่าตั้งแต่ 3-30 mCi  โดยเราต้องทำการแนะนำวิธีการปฏิบัติตัวหลังกลืนแร่ ดังนี้ 
อธิบายคนไข้ Low dose 🫧✨🥹
อธิบายการเตรียมตัวคนไข้ บอกว่าแต่ละคนกินเท่าไหร่ 
ให้เซ็นยินยอม 
และพามาดูห้องกินน้ำแร่  
2 สัปดาห์ถ้า 25 mCi 
1 สัปดาห์ 15-20 mCi ได้ ถ้าสำหรับผู้ใหญ่

อธิบายคนไข้ Low dose 🫧✨🥹
อธิบายการเตรียมตัวคนไข้ บอกว่าแต่ละคนกินเท่าไหร่ 
ให้เซ็นยินยอม 
และพามาดูห้องกินน้ำแร่  
2 สัปดาห์ถ้า 25 mCi 
1 สัปดาห์ 15-20 mCi ได้ ถ้าสำหรับผู้ใหญ่
🍦รังสีจะสลายไป ไม่ค้าง 
🍦ปัสสาวะ  ราดน้ำเยอะๆ 3-5 ครั้ง 
👅น้ำลาย จานชาม ผลไม้🍉
💧เหงื่อ อ